<a href="https://colab.research.google.com/github/domschl/torch-transformer-poet/blob/main/torch_transformer_poet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Torch-Transformer-Poet

Please review [ml-indie-tools](https://github.com/domschl/ml-indie-tools), a collection machine learning tools that provides support for more environment indepent code. It will access your Google Drive when using with Google Colab.

In [3]:
!pip install -U ml-indie-tools

In [4]:
import logging
import os
import sys
import copy
import json
import time
import datetime
import random
import numpy as np

import torch

In [5]:
from ml_indie_tools.env_tools import MLEnv
from ml_indie_tools.Gutenberg_Dataset import Gutenberg_Dataset
from ml_indie_tools.Text_Dataset import Text_Dataset

from ml_indie_tools.pytorch_custom_layers import MultiHeadSelfAttention

## Preliminary

A pytorch deep multi-head attention model for text generation following Andrej Karpathy's [video-lecture-ng](https://github.com/karpathy/ng-video-lecture/blob/master/gpt.py)

This code can use either CPU, GPU, or Apple Silicon. Google Colab is supported too, select the corresponding Colab runtime (menu: **`Runtime / Change runtime type`**)

## 0. Environment

In [6]:
cached_batch_data = None   # Do regenerate time-consuming training data, if aleady cached.

ml_env = MLEnv(platform='pt', accelerator='fastest')
ml_env.describe()

'OS: Darwin, Python: 3.10.8 (Conda), Jupyter Notebook Pytorch: 2.0.0.dev20230121, GPU: MPS Metal accelerator (system memory)'

In [7]:
project_name='women_writers'
model_name='ngpt_v1_pt'

# NOTICE: This will request access to Google Drive, if running on Google Colab. Google Drive is used to store snapshots
# training data. See project ml-indie-tools: https://github.com/domschl/ml-indie-tools 
#
# Note: you need to allow popups in your browser for COLAB, otherwise you won't see the google-drive login box, and drive access will fail!

root_path, project_path, model_path, data_path, log_path = ml_env.init_paths(project_name=project_name, model_name=model_name)

print(f"Root path (all projects) : {root_path} (This will be '.' (current dir) for local projects, and a google drive path for Colab)")
print(f"Project path             : {project_path} (Changes to the file system happen only below this project path")
print(f"Model path (snapshots)   : {model_path} (Model weights and snapshots are stored here)")
print(f"Data path (training data): {data_path} (Training data will be downloaded here)")
print(f"Log dir (tensorboard)    : {log_path} (it doesn't work to put logs on gdrive due to caching, hence local dir)")

Root path (all projects) : . (This will be '.' (current dir) for local projects, and a google drive path for Colab)
Project path             : . (Changes to the file system happen only below this project path
Model path (snapshots)   : ./model/ngpt_v1_pt (Model weights and snapshots are stored here)
Data path (training data): ./data (Training data will be downloaded here)
Log dir (tensorboard)    : ./logs (it doesn't work to put logs on gdrive due to caching, hence local dir)


##  1. Text library

`Text_Dataset` and `Gutenberg_Dataset` classes: libraries for training, 
encoding, batch generation, and formatted source display. It read some 
books from Project Gutenberg and supports creation of training batches. 
The output functions support highlighting to allow to compare generated 
texts with the actual sources to help to identify identical (memorized) 
parts.

In [8]:
use_dark_mode=False # Set to false for white background. HTML-text-compare uses background-colorization to identify different sources. Those background colors are dependent on the theme type.

In [9]:
logging.basicConfig(level=logging.INFO)
cache_dir = os.path.join(data_path, 'gutenberg_cache')
gd = Gutenberg_Dataset(cache_dir=cache_dir)

In [10]:
# sample searches
search_spec= {"author": ["Emily Brontë", "Jane Austen", "Virginia Woolf"], "language": ["english"]}

book_list=gd.search(search_spec)
book_cnt = len(book_list)
print(f"{book_cnt} matching books found with search {search_spec}.")
if book_cnt<40:
    # Note: please verify that book_cnt is 'reasonable'. If you plan to use a large number of texts, 
    # consider [mirroring Gutenberg](https://github.com/domschl/ml-indie-tools#working-with-a-local-mirror-of-project-gutenberg)
    book_list = gd.insert_book_texts(book_list, download_count_limit=book_cnt)  
else:
    logging.error("Please verify your book_list, a large number of books is scheduled for download. ABORTED.")

21 matching books found with search {'author': ['Emily Brontë', 'Jane Austen', 'Virginia Woolf'], 'language': ['english']}.


In [11]:
for i in range(len(book_list)):
    print(f"{i}: {book_list[i]['title']} - {book_list[i]['author']}, {book_list[i]['ebook_id']}")

0: The Common Reader - Virginia Woolf, 64457
1: Mr. Bennett and Mrs. Brown - Virginia Woolf, 63022
2: The Younger Sister, Volumes 1-3 - Catherine Anne Austen Hubback and Jane Austen, 54066
3: The Younger Sister, Vol. 3 - Catherine Anne Austen Hubback and Jane Austen, 54012
4: The Younger Sister, Vol. 2 - Catherine Anne Austen Hubback and Jane Austen, 54011
5: The Younger Sister, Vol. 1 - Catherine Anne Austen Hubback and Jane Austen, 54010
6: Pride and Prejudice - Jane Austen, 42671
7: The Letters of Jane Austen - Jane Austen, 42078
8: The Complete Project Gutenberg Works of Jane Austen - Jane Austen, 31100
9: Jacob's Room - Virginia Woolf, 5670
10: Pride and Prejudice - Jane Austen, 1342
11: Night and Day - Virginia Woolf, 1245
12: Love And Friendship And Other Early Works - Jane Austen, 1212
13: Lady Susan - Jane Austen, 946
14: Wuthering Heights - Emily Brontë, 768
15: Sense and Sensibility - Jane Austen, 161
16: Emma - Jane Austen, 158
17: The Voyage Out - Virginia Woolf, 144
18: M

In [12]:
MAX_TOKENS = 20000  # This becomes vocab_size
MAX_NGRAM_LEN = 10   # Max length of a token

select = ("Bennett", "1342", "5670", "1245", "161", "141", "121", "105", "Susan", "Wuthering", "Emma", "Voyage")  # List unique single-words from title or ebook_id to select a given book
sub_book_list = [book_list[i] for i in range(len(book_list)) if not set([book_list[i]['ebook_id']]+book_list[i]['title'].split(' ')).isdisjoint(set(select))]

print("Using:")
for i in range(len(sub_book_list)):
    print(f"{i+1}: {sub_book_list[i]['title']} - {sub_book_list[i]['author']}")

textlib_dataset = None  # Forces re-caching
td = Text_Dataset(sub_book_list)
td.init_tokenizer(tokenizer='ngram', max_ngrams=MAX_NGRAM_LEN, max_tokens=MAX_TOKENS)


INFO:Datasets:Loaded 12 texts
INFO:Datasets:Extracting ngrams of length 1..10 from text_list, selecting 20000 most used ngrams.


Using:
1: Mr. Bennett and Mrs. Brown - Virginia Woolf
2: Jacob's Room - Virginia Woolf
3: Pride and Prejudice - Jane Austen
4: Night and Day - Virginia Woolf
5: Lady Susan - Jane Austen
6: Wuthering Heights - Emily Brontë
7: Sense and Sensibility - Jane Austen
8: Emma - Jane Austen
9: The Voyage Out - Virginia Woolf
10: Mansfield Park - Jane Austen
11: Northanger Abbey - Jane Austen
12: Persuasion - Jane Austen


INFO:Datasets:Encoding text corpora as ngrams.
INFO:Datasets:Encoding text Mr. Bennett and Mrs. Brown...
INFO:Datasets:Encoding text Jacob's Room...
INFO:Datasets:Encoding text Pride and Prejudice...
INFO:Datasets:Encoding text Night and Day...
INFO:Datasets:Encoding text Lady Susan...
INFO:Datasets:Encoding text Wuthering Heights...
INFO:Datasets:Encoding text Sense and Sensibility...
INFO:Datasets:Encoding text Emma...
INFO:Datasets:Encoding text The Voyage Out...
INFO:Datasets:Encoding text Mansfield Park...
INFO:Datasets:Encoding text Northanger Abbey...
INFO:Datasets:Encoding text Persuasion...
INFO:Datasets:Encoding text corpora as ngrams done.


In [13]:
SEQUENCE_LEN = 64
# SUB_PROBABILITY = 0.15  # like BERT

td.init_getitem(sample_type='encoded', sample_length=SEQUENCE_LEN+1, content_stepping=1)

num_records = len(td)

print(f"{num_records} records")

1525331 records


In [14]:
def get_sample_batch(td, batch_size):
    # generate a small batch of data of inputs x and targets y
    # ix = torch.randint(len(data) - block_size, (batch_size,))
    # x = torch.stack([data[i : i + block_size] for i in ix])
    # y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    # x, y = x.to(device), y.to(device)
    # return x, y
    for i in range(batch_size):
        data = td.get_random_item()
        Xi = data[:-1]
        yi = data[1:]
        if i==0:
            # smpX=np.array(Xi, dtype=np.float32)
            smpX=np.array(Xi, dtype=np.int32)
            smpy=np.array(yi, dtype=np.int32)
        else:
            # smpX = np.vstack((smpX, np.array(Xi, dtype=np.float32)))
            smpX = np.vstack((smpX, np.array(Xi, dtype=np.int32)))
            smpy = np.vstack((smpy, np.array(yi, dtype=np.int32)))
    return np.array(smpX), np.array(smpy)

In [15]:
test_x, test_y = get_sample_batch(td, 2)
for i in range(len(test_x)):
    xi=[int(x) for x in test_x[i]]
    print(f"[{i}](l={len(xi)}): X=>{td.decode(xi)}<,\ny=>{td.decode(test_y[i])}<")

[0](l=64): X=>up her sister, and thinking
no more of the matter; but Mrs. Norris had a spirit of activity, which
could not be satisfied till she had written a long and angry letter to
Fanny, to point out the folly of her conduct, and threaten her with all
its possible ill consequences. Mrs. Price, in her turn, was injured and
angry; and an an<,
y=>her sister, and thinking
no more of the matter; but Mrs. Norris had a spirit of activity, which
could not be satisfied till she had written a long and angry letter to
Fanny, to point out the folly of her conduct, and threaten her with all
its possible ill consequences. Mrs. Price, in her turn, was injured and
angry; and an answer<
[1](l=64): X=>. “Does nothing occur to you?—Do not you recollect?—Harriet
Smith.”

Her cheeks flushed at the name, and she felt afraid of something,
though she knew not what.

“Have you heard from her yourself this morning?” cried he. “You have, I
believe, and know the whole.”

“No, I have not; I know nothing<,
y=>D

In [16]:
test_x.shape, test_y.shape

((2, 64), (2, 64))

## 2. data for texts

In [17]:
def expand_name_template(template, params):
    exp=copy.copy(template)
    for key in params:
        src="{"+key+"}"
        dst=f"{params[key]}"
        exp=exp.replace(src,dst).replace('[','(').replace(']',')')
    return exp

def save_model_metadata(epoch, suffix='std'):
    meta_file = os.path.join(model_path, f'model_meta_{suffix}.json')
    params['current_epoch'] = epoch
    try:
        with open(meta_file, 'w') as f:
            f.write(json.dumps(params))
    except Exception as e:
        print(f"Failed to store model metadata at {model_path}: {e}")
        return False
    return True

def read_model_metadata(suffix="std"):
    meta_file = os.path.join(model_path, f'model_meta_{suffix}.json')
    try:
        with open(meta_file, 'r') as f:
            meta = json.load(f)
    except Exception as e:
        print(f"Cannot access project meta-data at {meta_file}: {e}, starting anew.")
        return None
    return meta

def is_metadata_compatible(params, meta):
    is_valid=True
    keys=set(list(params.keys())+list(meta.keys()))
    for key in keys:
        if key in updatable_keys:
            continue
        if key not in meta:
            print(f"Key {key} not available in last checkpoint model_meta, params[{key}]: {params[key]}, cannot import incompatible model. Put key in `updatable_keys` list, if irrelevant.")
            is_valid = False
        elif key not in params:
            print(f"Key {key} not available in params, last checkpoint model_meta[{key}]: {meta[key]}, cannot import incompatible model. Put key in `updatable_keys` list, if irrelevant.")
            is_valid = False
        elif meta[key]!=params[key]:
            print(f"Last checkpoint model_meta[{key}]: {meta[key]} != params[{key}]: {params[key]}, cannot import incompatible model. Put key in `updatable_keys` list, if irrelevant.")
            is_valid = False
    if is_valid is False:
        print("Aborting import.")
        return False
    return True

In [18]:
vocabulary_size = td.get_unique_token_count()  # vocabulary-size

lyrs = 6;

params = { # Multi-head self-attention
    'name': '{mhsa_layers}x{heads}x{units}x{vocab_size}',

    'mhsa_layers': lyrs, 
    'heads': 12, # [16]*lyrs,
    # 'units': [512]*lyrs,
    # 'norm': 'softmax', # this is for within each head (all-you-need: softmax)
    # 'mh_normalize': True,  # use layer-norm after concatenation (or additiona) of the heads
    'causal': True,  # Use causal self-attention
    # 'l2_regularizer': 1e-9,
    'dropout': 0.1,       # no dropout: 0.0
    # 'join_heads_by_add': True,  # stragegy how multi-heads are joined: False: concat (as in all-you-need), True: relu+add of all the heads: less resources, similar perf.
    'vocab_size': vocabulary_size,
    'sequence_len': SEQUENCE_LEN,
    'embedding_size': 384,
    'test_iterations': 10,  # number of epocs for loss estimation

    'batch_size': 64,
    'learning_rate': 0.002,
    # 'clipvalue': None,
    'sample_every_n_iterations': 100,
    
    'max_iterations': 1000000  # maximum number of training iterations
}

# if len(params['heads'])!=params['mhsa_layers']: # or len(params['units'])!=params['mhsa_layers']:
#     print("ERROR: lenght of 'heads' and 'units' must be equal to mhsa_layers!")
    
model_suffix = expand_name_template(params['name'], params)
# Put 'important' params in checkpoint-pathname to separate model-data:
checkpoint_dir = os.path.join(model_path, f"training_checkpoints_{model_suffix}")
if os.path.exists(checkpoint_dir) is False:
    os.makedirs(checkpoint_dir)

# When comparing if training-data is compatible with new params set, 
# the following keys are updatable, they can be changed while continuing
# to use existing checkpoints and continue training with those values
# changed:
updatable_keys=['learning_rate', 'batch_size', 'current_epoch', 'dropout', 
             'sample_every_n_epochs']

# These values are taking from saved checkpoint:
keep_keys=['current_epoch']

continue_last = True
if continue_last is False:
    print("NOT continuing based on existing training! New start.")

meta = read_model_metadata(suffix=model_suffix)
if meta is not None and is_metadata_compatible(params, meta) is True and continue_last is True:
    for key in keep_keys:
        if key in meta:
            params[key]=meta[key]
    if params is not None:
        print(f"Continuing last session from epoch {params['current_epoch']}")
    else:
        print(f"No previous data, starting new model")
else:
    print("Starting new model")

print(params)

Cannot access project meta-data at ./model/ngpt_v1_pt/model_meta_6x12x{units}x20000.json: [Errno 2] No such file or directory: './model/ngpt_v1_pt/model_meta_6x12x{units}x20000.json', starting anew.
Starting new model
{'name': '{mhsa_layers}x{heads}x{units}x{vocab_size}', 'mhsa_layers': 6, 'heads': 12, 'causal': True, 'dropout': 0.1, 'vocab_size': 20000, 'sequence_len': 64, 'embedding_size': 384, 'test_iterations': 10, 'batch_size': 64, 'learning_rate': 0.002, 'sample_every_n_iterations': 100, 'max_iterations': 1000000}


In [19]:
num_batches = num_records // params['batch_size']
print(f"num_batches = {num_batches}")

num_batches = 23833


In [20]:
def get_torch_batch(td, batch_size, device, split=None):
    x, y = get_sample_batch(td, batch_size)
    return torch.tensor(x, dtype=torch.long).to(device), torch.tensor(y, dtype=torch.long).to(device)

In [21]:
get_torch_batch(td, 2, 'cpu')

(tensor([[ 6099, 17073, 10411, 16071,  2146,  6080,  3706,  3142,  2649,  8387,
          16438,  3558,  9830, 12973,  1287, 18664,   494,  7806, 17306,  4944,
           5022,  2934, 12165,   456,   471, 13775,  6967,  3147, 19494,   588,
          15078,  9436,   360,  5890,  2757,  2682,  1649,  5101, 10755,   206,
           2514,  8462,   479,  2795,  9309,  7768,  9996, 10890,  3560,   803,
          15225,  6113,  5507, 12868, 11585,  6471,  2789,  1151,  8577,  1153,
           1321,   595, 13917,   507],
         [ 5743, 14388,  8043,    24,  6827,  2352,  1129,  7020,  1745,  8854,
            779, 15600,  1536, 13965, 17682,  1161, 13976, 11937,  1027,  1330,
          11134,   757,  3460,   370,  5714,  5944,    41,  6412, 18554, 13626,
          14246,  1548,   965,   535, 10314,  6242,  3394,  6837,  7566,  3369,
          12466,  2641,  9653, 12543,  9630, 16414,  1839, 12466,  2907,  7909,
          15083,  1539, 13510, 14171, 12202,  2594,  1088,  5625,  5903,  1811,
 

In [22]:
@torch.no_grad()
def estimate_loss(device):
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(params['test_iterations'])
        for k in range(params['test_iterations']):
            print(".", end="", flush=True)
            X, Y = get_torch_batch(td, params['batch_size'], device, split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    print("]\r", end="", flush=True)
    return out


def generate_sample(td, toks=100):
    # generate from the model
    context = torch.zeros((1, 1), dtype=torch.long, device=device)
    print(td.decode(model.generate(context, max_new_tokens=toks)[0].tolist()))
    # open('more.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))


# @torch.compile
def do_train_step(xb, yb):
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

In [23]:
# XXX!
device = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device("mps") if torch.backends.mps.is_available() else device

In [ ]:
print("creating model...")
model_cpu = MultiHeadSelfAttention(params['vocab_size'], params['embedding_size'], 
                                   params['sequence_len'], params['dropout'], 
                                   params['heads'], params['mhsa_layers'], params['causal'], device)
model = model_cpu.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in model.parameters()) / 1e6, "M parameters")
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=params['learning_rate'])

dt0 = time.time()
print("training...")
for iter in range(params['max_iterations']):
    print(f"Iteration: {iter+1:5d}", end="\r", flush=True)
    # every once in a while evaluate the loss on train and val sets
    if (iter + 1) % params['sample_every_n_iterations'] == 0 or iter == params['max_iterations'] - 1:
        dt = time.time()
        print(f"loss eval)[", end="", flush=True)
        losses = estimate_loss(device)
        print(
            f"step {iter+1}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}, time {(dt-dt0)/params['sample_every_n_iterations']:.3f} sec/iter"
        )
        print("Sample: ", end="", flush=True)
        generate_sample(td)
        dt0 = time.time()
    # sample a batch of data
    xb, yb = get_torch_batch(td, params['batch_size'], device, "train")

    # evaluate the loss
    do_train_step(xb, yb)

creating model...
26.045216 M parameters
training...
step 100: train loss 8.9379, val loss 8.9468, time 0.761 sec/iter
Sample: <unk>it to ; and ellcaI had with hermeandlemake hoooff lliIn, sncvis; and the interigion, and oneso muchach other
in , as ere btion and the worgreealk xt,
. His ls ought to , that , that hairly. Noon with for in to be
 the preand toss w, as she pasitudeFairfaxuihimalsher seventielised by the ba; thing the ind ked you will o d in a
moteewn ; but s is ot auneall daughteralsdoororEltonwellas sheis own lace. I Cad to x’ve mentngameappinesss hait would throughlowensenight ng. 
